In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import product
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
CSV_PATH = "team_differences_performance.csv"  # change if needed
RANDOM_STATE = 42

In [ ]:
# --- Expanded Hyperparameters to search/tune ---
HP_GRID = {
    "learning_rate": [1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 3e-2],
    "epochs": [300, 600, 1000, 2000],
    "batch_size": [8, 16, 32, 64, 128],
    "l2_lambda": [0.0, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
}

In [15]:
# --- Utilities ---
def sigmoid(z: np.ndarray) -> np.ndarray:
    z = np.clip(z, -500, 500)
    return 1.0 / (1.0 + np.exp(-z))

def predict_proba(X: np.ndarray, w: np.ndarray, b: float) -> np.ndarray:
    return sigmoid(X @ w + b)

def predict_label(X: np.ndarray, w: np.ndarray, b: float, thr: float = 0.5) -> np.ndarray:
    return (predict_proba(X, w, b) >= thr).astype(int)

def accuracy(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    return (y_true == y_pred).mean()

def train_logreg_minibatch(
    X: np.ndarray,
    y: np.ndarray,
    learning_rate: float,
    epochs: int,
    batch_size: int,
    l2_lambda: float,
    w_init: np.ndarray | None = None,
    b_init: float | None = None,
    rng: np.random.Generator | None = None,
) -> tuple[np.ndarray, float]:
    """
    Train logistic regression with mini-batch gradient descent.
    L2 penalty applied to weights (not bias).
    """
    n, d = X.shape
    rng = rng or np.random.default_rng(RANDOM_STATE)
    w = np.zeros(d) if w_init is None else w_init.copy()
    b = 0.0 if b_init is None else float(b_init)

    idx = np.arange(n)
    for _ in range(epochs):
        rng.shuffle(idx)
        for start in range(0, n, batch_size):
            batch = idx[start:start + batch_size]
            Xb = X[batch]
            yb = y[batch]
            pb = predict_proba(Xb, w, b)

            m = yb.shape[0]
            # gradients (average over batch)
            grad_w = (Xb.T @ (pb - yb)) / m + l2_lambda * w
            grad_b = np.mean(pb - yb)

            # update
            w -= learning_rate * grad_w
            b -= learning_rate * grad_b

    return w, b

In [18]:
# --- Main workflow ---
def main():
    # Load dataset
    df = pd.read_csv(CSV_PATH)

    # Target & features
    y = df["playoffs"]
    if y.dtype == bool:
        y = y.astype(int)
    elif y.dtype == object:
        y = (
            y.astype(str).str.strip().str.lower()
             .map({"yes": 1, "no": 0, "true": 1, "false": 0, "1": 1, "0": 0})
             .astype(int)
        )
    else:
        y = y.astype(int)

    X = df.drop(columns=["playoffs"]).select_dtypes(include=["number"])

    if "season" in X.columns:
        X = X.drop(columns=["season"])

    # 60/20/20 split (stratified)
    X_train_full, X_temp, y_train_full, y_temp = train_test_split(
        X, y, test_size=0.40, random_state=RANDOM_STATE, stratify=y
    )
    X_val_full, X_test_full, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.50, random_state=RANDOM_STATE, stratify=y_temp
    )

    # Scale (fit on train only)
    scaler = StandardScaler().fit(X_train_full)
    X_train = scaler.transform(X_train_full)
    X_val = scaler.transform(X_val_full)
    X_test = scaler.transform(X_test_full)

    # Convert to numpy
    X_train = np.asarray(X_train, dtype=float)
    X_val = np.asarray(X_val, dtype=float)
    X_test = np.asarray(X_test, dtype=float)
    y_train_np = np.asarray(y_train_full.values, dtype=int)
    y_val_np = np.asarray(y_val.values, dtype=int)
    y_test_np = np.asarray(y_test.values, dtype=int)

    rng = np.random.default_rng(RANDOM_STATE)

    # Hyperparameter search on TRAIN; evaluate on VAL
    best_acc = -1.0
    best_hp = None
    best_w, best_b = None, None

    hp_keys = list(HP_GRID.keys())
    for values in product(*[HP_GRID[k] for k in hp_keys]):
        hp = dict(zip(hp_keys, values))

        w, b = train_logreg_minibatch(
            X_train, y_train_np,
            learning_rate=hp["learning_rate"],
            epochs=hp["epochs"],
            batch_size=hp["batch_size"],
            l2_lambda=hp["l2_lambda"],
            rng=rng,
        )

        val_pred = predict_label(X_val, w, b)
        val_acc = accuracy(y_val_np, val_pred)

        if val_acc > best_acc:
            best_acc = val_acc
            best_hp = hp
            best_w, best_b = w.copy(), b

    print(f"Best validation accuracy: {best_acc:.4f}")
    print(f"Best GD hyperparameters: {best_hp}")

    # Retrain from scratch on TRAIN+VAL with best HPs, then test
    X_combined_full = np.vstack([X_train, X_val])
    y_combined = np.concatenate([y_train_np, y_val_np])

    w_final, b_final = train_logreg_minibatch(
        X_combined_full, y_combined,
        learning_rate=best_hp["learning_rate"],
        epochs=best_hp["epochs"],
        batch_size=best_hp["batch_size"],
        l2_lambda=best_hp["l2_lambda"],
        rng=rng,
    )

    test_pred = predict_label(X_test, w_final, b_final)
    test_acc = accuracy(y_test_np, test_pred)
    print(f"Test accuracy: {test_acc:.4f}")

    # Simple confusion matrix
    cm = np.zeros((2, 2), dtype=int)
    for yt, yp in zip(y_test_np, test_pred):
        cm[yt, yp] += 1
    print("Confusion matrix [[TN, FP], [FN, TP]]:")
    print(cm.tolist())

    # Additional metrics
    precision = precision_score(y_test_np, test_pred, average='macro')
    recall = recall_score(y_test_np, test_pred, average='macro')
    f1 = f1_score(y_test_np, test_pred, average='macro')
    roc_auc = roc_auc_score(y_test_np, predict_proba(X_test, w_final, b_final))

    print(f"Precision (macro): {precision:.3f}")
    print(f"Recall (macro): {recall:.3f}")
    print(f"F1 (macro): {f1:.3f}")
    print(f"ROC–AUC: {roc_auc:.3f}")

if __name__ == "__main__":
    main()

Best validation accuracy: 0.4581
Best GD hyperparameters: {'learning_rate': 0.0001, 'epochs': 300, 'batch_size': 8, 'l2_lambda': 0.0}
Test accuracy: 0.4581
Confusion matrix [[TN, FP], [FN, TP]]:
[[71, 0], [84, 0]]


c:\Users\thepa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


ValueError: Input contains NaN.